In [1]:
from gevent import monkey
monkey.patch_all()
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = = = = =

import pandas as pd
from selenium.webdriver import ChromeOptions

option = ChromeOptions()
option.add_experimental_option('useAutomationExtension', False)
option.add_argument('blink-settings=imagesEnabled=false')
option.add_argument('--disable-blink-features=AutomationControlled')
option.add_argument('--disable-dev-shm-usage')
option.add_argument('--disable-gpu')
option.add_experimental_option('excludeSwitches', ['enable-automation'])
# option.add_argument('--headless')
option.add_argument('--no-sandbox')
option.add_argument('--proxy-server=http://u431.kdltps.com:15818')
option.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36')
option.page_load_strategy = 'eager'

from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By

# = = = = = = = = = = = = = = = = = =

df_menu = pd.read_excel('./menu.xlsx', header=0)
list_url = df_menu['Url'].to_list()

print('总数量：' + str(len(list_url)))
print()

work = Queue()
for url in list_url:
    work.put_nowait(url)

# = = = = = = = = = = = = = = = = = =

df = pd.DataFrame(columns=['Url',
                           'Title',
                           'mpn',
                           'Brand',
                           'Part_Number',
                           'UPC',
                           'Vehicle',
                           'Src',
                           'status'])

# = = = = = = = = = = = = = = = = = =

def crawler():
    global df
    
    while not work.empty():
        url = work.get_nowait()
        
        status = 'error'
        for _ in range(31):
            try:
                browser = Chrome(service=Service('../../../chromedriver'), options=option)
                browser.maximize_window()
                
                # = = = = = = = = = = = = = = = = = =

                browser.get(url)
                
                time.sleep(5)
                
                # = = = = = = = = = = = = = = = = = =
                
                wait = WebDriverWait(browser, 60)
                wait.until(EC.presence_of_element_located((By.XPATH, '//div[@class="prod-title js-prod-title"]/h1[@class="name"]')))
                title = browser.find_element(by=By.XPATH, value='//div[@class="prod-title js-prod-title"]/h1[@class="name"]').text

                # = = = = = = = = = = = = = = = = = =
                
                list_row = browser.find_element(by=By.XPATH, value='//div[@class="prod-title js-prod-title"]').text.split('|')
                mpn = ''
                for row in list_row:
                    if row.startswith('Item # mpn'):
                        mpn = row.split('Item # mpn')[1]

                # = = = = = = = = = = = = = = = = = =
                
                wait.until(EC.presence_of_element_located((By.XPATH, '//div[@class="prod_offer"]')))
                list_offer = browser.find_elements(by=By.XPATH, value='//div[@class="prod_offer"]/div[@class="prod-offer-item"]')

                # = = = = = = = = = = = = = = = = = =
                
                brand = ''
                part_number = ''
                upc = ''
                for offer in list_offer:
                    if offer.text.startswith('Brand:'):
                        brand = offer.text.split('Brand:')[1]
                    elif offer.text.startswith('Part Number:'):
                        part_number = offer.text.split('Part Number:')[1]
                    elif offer.text.startswith('UPC:'):
                        upc = offer.text.split('UPC:')[1]

                # = = = = = = = = = = = = = = = = = =
                
                js = '''let list_expand = document.querySelectorAll('div[class="mmy-row"]');
                for(let i=0; i<list_expand.length; i++) {
                        list_expand[i].getElementsByClassName('mmy-row-expand')[0].click();
                }'''

                browser.execute_script(js)

                # = = = = = = = = = = = = = = = = = =
                
                wait.until(EC.presence_of_element_located((By.XPATH, '//div[@class="prod-section-content"]')))
                list_row = browser.find_elements(by=By.XPATH, value='//div[@class="prod-section-content"]')

                list_vehicle = []
                for row in list_row:
                    if row.text.startswith('Vehicle Fitment'):
                        list_make = row.find_elements(by=By.XPATH, value='./descendant::div[@class="mmy-row-title pointer js-mmy-row-expand"]')
                        for make in list_make:
                            list_vehicle += [make.text + ' ' + model_year.text for model_year in make.find_elements(by=By.XPATH, value='./following-sibling::div[@class="mmy-row-description"]/descendant::div[@class="mmy-subrow-title"]')]

                vehicle = '\n'.join(list_vehicle)

                # = = = = = = = = = = = = = = = = = =
                
                wait.until(EC.presence_of_element_located((By.XPATH, '//a[@class="product-image-main"]')))
                src = browser.find_element(by=By.XPATH, value='//a[@class="product-image-main"]').get_attribute('href')

                # = = = = = = = = = = = = = = = = = =
                
                status = 'ok'
                df_temp = pd.DataFrame([{'Url': url,
                                         'Title': title,
                                         'mpn': mpn,
                                         'Brand': brand,
                                         'Part_Number': part_number,
                                         'UPC': upc,
                                         'Vehicle': vehicle,
                                         'Src': src,
                                         'status': 'ok'}])
                
                # = = = = = = = = = = = = = = = = = =

                list_row = [row.text for row in browser.find_elements(by=By.XPATH, value='//table[@class="spec-stripped-table"]/tbody/tr/td')]

                start = ''
                for row in list_row:
                    if ':' in row:
                        name_value = row.split(': ')
                        df_temp[(start + ' ' + name_value[0]).strip().lower()] = ': '.join(name_value[1:])
                    else:
                        start = row
                        
                browser.quit()

                break

            except:
                time.sleep(3)
                continue

        # = = = = = = = = = = = = = = = = = =

        if status == 'error':
            df_temp = pd.DataFrame([{'Url': url,
                                     'status': 'error'}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')
        
        print(url +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))
        
# = = = = = = = = = = = = = = = = = =

list_task = []
for _ in range(9):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df.to_excel('./part.xlsx', index=False)
print()
print('搞定')

总数量：947



KeyboardInterrupt
2023-07-10T07:35:36Z


KeyboardInterrupt: 